# Mean and Standard Deviation by Spark's combineByKey()

In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="RDDMeanStdev")

23/03/08 13:15:40 WARN Utils: Your hostname, mgadi-ThinkPad-T470s-W10DG resolves to a loopback address: 127.0.1.1; using 192.168.1.35 instead (on interface wlp58s0)
23/03/08 13:15:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/08 13:15:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
a = [("A", 2.), ("A", 4.), ("A", 9.),("B", 10.), ("B", 20.),("C", 3.), ("C", 5.), ("C", 8.), ("C", 12.)]

In [3]:
a

[('A', 2.0),
 ('A', 4.0),
 ('A', 9.0),
 ('B', 10.0),
 ('B', 20.0),
 ('C', 3.0),
 ('C', 5.0),
 ('C', 8.0),
 ('C', 12.0)]

In [4]:
rdd = sc.parallelize( [("A", 2.),("A", 4.),("A", 9.),("B", 10.),("B", 20.),("C", 3.),("C", 5.),("C", 8.),("C", 12.)])

In [5]:
rdd.collect()

[('A', 2.0),
 ('A', 4.0),
 ('A', 9.0),
 ('B', 10.0),
 ('B', 20.0),
 ('C', 3.0),
 ('C', 5.0),
 ('C', 8.0),
 ('C', 12.0)]

In [6]:
rdd.count()

9

## Mean by Key with combineByKey()

In [20]:
rdd.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).collect()

[('A', (15.0, 3)), ('B', (30.0, 2)), ('C', (28.0, 4))]

In [21]:
sumCount = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [24]:
res = sumCount.collect()

In [32]:
res

[('A', (15.0, 3)), ('B', (30.0, 2)), ('C', (28.0, 4))]

In [31]:
res[0][1][0]/res[0][1][1]

5.0

In [34]:
type(sumCount)

pyspark.rdd.PipelinedRDD

In [40]:
sumCount.map(lambda x:(x[0],x[1][0]/x[1][1])).take(10)

[('A', 5.0), ('B', 15.0), ('C', 7.0)]

In [42]:
sumCount.mapValues(lambda x:x[0]/x[1]).take(10)

[('A', 5.0), ('B', 15.0), ('C', 7.0)]

In [43]:
mean = sumCount.mapValues(lambda x : x[0]/x[1])

In [44]:
mean.collect()

[('A', 5.0), ('B', 15.0), ('C', 7.0)]

## Merging by Key

In [46]:
type(mean)

pyspark.rdd.PipelinedRDD

In [47]:
rdd.take(10)

[('A', 2.0),
 ('A', 4.0),
 ('A', 9.0),
 ('B', 10.0),
 ('B', 20.0),
 ('C', 3.0),
 ('C', 5.0),
 ('C', 8.0),
 ('C', 12.0)]

In [48]:
mean.take(10)

[('A', 5.0), ('B', 15.0), ('C', 7.0)]

In [49]:
rdd_mean = rdd.join(mean)

In [50]:
rdd_mean.take(10)

[('B', (10.0, 15.0)),
 ('B', (20.0, 15.0)),
 ('C', (3.0, 7.0)),
 ('C', (5.0, 7.0)),
 ('C', (8.0, 7.0)),
 ('C', (12.0, 7.0)),
 ('A', (2.0, 5.0)),
 ('A', (4.0, 5.0)),
 ('A', (9.0, 5.0))]

In [51]:
rdd_mean.map(lambda x : x).collect()

[('B', (10.0, 15.0)),
 ('B', (20.0, 15.0)),
 ('C', (3.0, 7.0)),
 ('C', (5.0, 7.0)),
 ('C', (8.0, 7.0)),
 ('C', (12.0, 7.0)),
 ('A', (2.0, 5.0)),
 ('A', (4.0, 5.0)),
 ('A', (9.0, 5.0))]

\begin{equation}\label{stdev}
s^2 = \sqrt{{\sum_{i=1}^n (y_i - \bar{y})^2}/N}.
\end{equation}

In [52]:
import math

In [53]:
(2-5)**2 + (4-5)**2 + (9-5)**2

26

In [54]:
((2-5)**2 + (4-5)**2 + (9-5)**2)

26

In [69]:
math.sqrt(((2-5)**2 + (4-5)**2 + (9-5)**2)/3)

2.943920288775949

In [56]:
import math
def  stdDev( x, mean):
    stdDeviation = (x - mean)**2
    return (mean, stdDeviation)

In [59]:
rdd_mean.mapValues(lambda x:(x[0]-x[1])**2).reduceByKey(lambda x,y:x+y).take(10)

[('B', 50.0), ('C', 46.0), ('A', 26.0)]

In [60]:
StdDev = rdd_mean.mapValues(lambda x : ((x[0]- x[1])**2))

In [61]:
StdDev.collect()

[('B', 25.0),
 ('B', 25.0),
 ('C', 16.0),
 ('C', 4.0),
 ('C', 1.0),
 ('C', 25.0),
 ('A', 9.0),
 ('A', 1.0),
 ('A', 16.0)]

In [62]:
StdDev = StdDev.reduceByKey(lambda x, y : x + y)

In [63]:
StdDev = StdDev.reduceByKey(lambda x, y: x + y)

In [64]:
StdDev.collect()

[('B', 50.0), ('C', 46.0), ('A', 26.0)]

In [65]:
sumCount.take(10)

[('A', (15.0, 3)), ('B', (30.0, 2)), ('C', (28.0, 4))]

In [66]:
sumCount.join(StdDev).collect()

[('A', ((15.0, 3), 26.0)), ('C', ((28.0, 4), 46.0)), ('B', ((30.0, 2), 50.0))]

In [67]:
mean_stddev = sumCount.join(StdDev)

In [68]:
mean_stddev.collect()

[('A', ((15.0, 3), 26.0)), ('C', ((28.0, 4), 46.0)), ('B', ((30.0, 2), 50.0))]

In [73]:
import numpy as np
mean_stddev.mapValues(lambda x:x[1]/x[0][1]).mapValues(lambda x:np.sqrt(x)).take(10)

[('A', 2.943920288775949), ('C', 3.391164991562634), ('B', 5.0)]

In [74]:
mean_stddev.map(lambda x:(x[0],x[1][0][0],x[1][0][1],x[1][1])).collect()

[('A', 15.0, 3, 26.0), ('C', 28.0, 4, 46.0), ('B', 30.0, 2, 50.0)]

In [75]:
mean_stddev.map(lambda x:(x[0],x[1][0][0]/x[1][0][1],x[1][1]/x[1][0][1])).collect()

[('A', 5.0, 8.666666666666666), ('C', 7.0, 11.5), ('B', 15.0, 25.0)]

In [76]:
mean_stddev.map(lambda x:(x[0],x[1][0][0]/x[1][0][1],math.sqrt(x[1][1]/x[1][0][1]))).collect()

[('A', 5.0, 2.943920288775949),
 ('C', 7.0, 3.391164991562634),
 ('B', 15.0, 5.0)]

---

# In one go

https://github.com/mahmoudparsian/pyspark-tutorial/blob/master/tutorial/combine-by-key/standard_deviation_by_combineByKey.md

In [77]:
python_list_of_tuples = [
    ("A", 2.), ("A", 4.), ("A", 9.),
    ("B", 10.), ("B", 20.),
    ("Z", 3.), ("Z", 5.), ("Z", 8.), ("Z", 12.)
]

In [78]:
rdd = sc.parallelize( python_list_of_tuples )

In [79]:
rdd.collect()

[('A', 2.0),
 ('A', 4.0),
 ('A', 9.0),
 ('B', 10.0),
 ('B', 20.0),
 ('Z', 3.0),
 ('Z', 5.0),
 ('Z', 8.0),
 ('Z', 12.0)]

In [80]:
rdd.count()

9

In [81]:
sumCount = rdd.combineByKey(lambda value: (value, value*value, 1),
                            lambda x, value: (x[0] + value, x[1] + value*value, x[2] + 1),
                            lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2])
                           )

In [82]:
sumCount.collect()

[('A', (15.0, 101.0, 3)), ('B', (30.0, 500.0, 2)), ('Z', (28.0, 242.0, 4))]

In [83]:
import math
def  stdDev( sumX, sumSquared, n ):
    mean = sumX / n
    stdDeviation = math.sqrt ((sumSquared - n*mean*mean) /n)
    return (mean, stdDeviation)

In [84]:
meanAndStdDev = sumCount.mapValues(lambda x : stdDev(x[0], x[1], x[2]))

In [85]:
meanAndStdDev.collect()

[('A', (5.0, 2.943920288775949)),
 ('B', (15.0, 5.0)),
 ('Z', (7.0, 3.391164991562634))]